In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
#from tkinter import Tk, filedialog
from skimage import img_as_ubyte
from skimage.segmentation import clear_border
from skimage import measure
import matplotlib.pyplot as plt
from keras.utils import normalize
import shutil
import pandas as pd
import pyvips
import re
import statistics
from tqdm import tqdm
logger = tf.get_logger()
logger.setLevel('ERROR')
# Image sizes need to be divisible by 16
IMG_HEIGHT = 688
IMG_WIDTH  = 432
coordinates = (250, 400, 700, 1100)  # Specify the ROI coordinates (left, upper, right, lower)

In [2]:
# Colab import
file_path = "documents/Scan1/Scan1_00000000_0000.tif"

file_dir, file_name = os.path.split(file_path)

name_numberAngPos = file_name.split('_0')[0] + "_0"

# Determination of the number of images per angular position
numberAngPos = len([f for f in os.listdir(file_dir) if f.startswith(name_numberAngPos) and f.endswith("_0000.tif")]) - 1

# Determination of the total number of projection images
numberProjIm = len([f for f in os.listdir(file_dir) if f.startswith(name_numberAngPos + "0000000_") and f.endswith(".tif")]) - 1


# Define Image Data List with Keywords
# The entire image is considered the ROI
imageData = {
    'filename': file_name,
    'path': file_dir + "/",
    'angPos': numberAngPos,
    'projPerAng': numberProjIm,
}


#return imageData, ImageROI
print(imageData)


{'filename': 'Scan1_00000000_0000.tif', 'path': 'documents/Scan1/', 'angPos': 904, 'projPerAng': 69}


In [4]:

def generate_image_filenames(max_num1, max_num2):
    for num1 in range(max_num1 + 1):
        for num2 in range(max_num2+ 1):
            filename = f"Scan1_{num1:08d}_{num2:04d}.tif"
            filenames.append(filename)
    return filenames



In [5]:
def load_batch(image_paths):
    images = []
    for image_path in image_paths:
        image = pyvips.Image.new_from_file(image_path, access='sequential')
        memory_buffer = image.write_to_memory()
        # Convert the memory buffer to a NumPy array
        height, width = image.height, image.width
        np_image = np.frombuffer(memory_buffer, dtype=np.uint16).reshape(height, width)
        images.append(np_image)
        isinstance(np_image, np.ndarray)
    return images

In [6]:
def preprocess_images(batch_images):
    preprocessed_images = []
    for image in batch_images:
        image = image[coordinates[1]:coordinates[3], coordinates[0]:coordinates[2]]
        # Resize the image using OpenCV
        resized_image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        # Append to the list of preprocessed images
        preprocessed_images.append(resized_image)
    #maybe change to -1
    preprocessed_images = np.expand_dims(normalize(np.array(preprocessed_images), axis=1),3)
    print(np.shape(preprocessed_images))
    return preprocessed_images



In [7]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

################################################################
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if inputs are normalized beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

    return model


In [8]:
IMG_CHANNELS = 1

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model = get_model()
model.load_weights('documents/dropletonly.hdf5')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 688, 432, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 688, 432, 16  160         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 688, 432, 16  0           ['conv2d[0][0]']                 
                                )                                                             

In [9]:
def post_processing(predictions):
    # Post-processing
    processed_images = []
    for prediction in predictions:
        # Binarize
        categoryImg_water = (prediction >= 0.3).astype(np.uint8)

        # Calculate buffer size (e.g., min dimension divided by 10)
        #buffer_size = min(categoryImg_water.shape) // 10
        #print(buffer_size)

        # Despeckling and clearing border
        #categoryImg_water_desp = clear_border(categoryImg_water, buffer_size=buffer_size)
        categoryImg_water_desp = categoryImg_water
        # Display despeckled image

        # Morphological operations
        se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
        categoryImg_water_morph = cv2.morphologyEx(categoryImg_water_desp.astype(np.uint8), cv2.MORPH_OPEN, se)
        categoryImg_water_morph = cv2.morphologyEx(categoryImg_water_morph, cv2.MORPH_CLOSE, se)


        # Append to processed_images list
        processed_images.append(categoryImg_water)

    return processed_images


In [10]:
def get_regionprops(processed_images, min_area=600):

    for processed_image in processed_images:
        # Label the connected components
        labeled_droplets, _ = measure.label(processed_image, return_num=True, connectivity=2)
        # Find region properties
        regions = measure.regionprops(labeled_droplets)

        # Filter regions based on area
        filtered_regions = [region for region in regions if region.area >= min_area]

        # Find the lowest droplet among filtered regions
        if filtered_regions:
            # Select the droplet with the maximum vertical position (row component of centroid)
            lowest_droplet = max(filtered_regions, key=lambda region: region.centroid[0])

            # Create a binary image with just the lowest droplet
            test_img = np.zeros_like(processed_image, dtype=np.uint8)
            for coord in lowest_droplet.coords:
                test_img[coord[0], coord[1]] = 1
            lowest_droplet_props = {
                'centroid': lowest_droplet.centroid,
                'area': lowest_droplet.area,
                'bbox': lowest_droplet.bbox,
                'filled_area': lowest_droplet.filled_area
            }

        else:
            # Set default values if no droplet meets the criteria
            lowest_droplet_props = {
                'centroid': (0, 0),
                'area': 0,
                'bbox': (0, 0, 0, 0),
                'filled_area': 0
            }
        props[len(props) + 1] = lowest_droplet_props


In [11]:
filenames = []
image_filenames = generate_image_filenames(imageData['angPos'], imageData['projPerAng'])
batch_size = 16

if 'props' in locals():
    del props
props = {}
n = len(image_filenames)
for i in tqdm(range(0, n, batch_size), total=n // batch_size, desc="Processing batches"):
    batch_filenames = image_filenames[i:i + batch_size]
    batch_image_paths = [os.path.join(imageData['path'], filename) for filename in batch_filenames]

    # Load the batch
    batch_images = load_batch(batch_image_paths)

    # Preprocess the batch
    preprocessed_batch_images = preprocess_images(batch_images)

    # Predict images
    predictions = model.predict_on_batch(preprocessed_batch_images)
    
    
    processed_images = post_processing(predictions)

    get_regionprops(processed_images)

Processing batches:   0%|          | 0/3959 [00:00<?, ?it/s]

(16, 688, 432, 1)


Processing batches:   0%|          | 1/3959 [00:10<11:41:26, 10.63s/it]

(16, 688, 432, 1)


Processing batches:   0%|          | 2/3959 [00:12<5:42:40,  5.20s/it] 

...

Processing batches: 100%|█████████▉| 3958/3959 [1:10:15<00:01,  1.00s/it]

(16, 688, 432, 1)


Processing batches: 100%|██████████| 3959/3959 [1:10:16<00:00,  1.02s/it]

(6, 688, 432, 1)


Processing batches: 3960it [1:10:18,  1.07s/it]                          


In [ ]:
#TEST
df = pd.DataFrame(props)
print(df)

In [13]:
#Add filenames to dictionary
pattern = r"Scan\d+_(\d+)_(\d+)\.tif"
new_props = {key: {**props[key], 'filename': filename, 'AngPos': int(re.search(pattern, filename).group(1)), 'ProjperAng': int(re.search(pattern, filename).group(2))} for key, filename in zip(props.keys(), filenames)}

#Calculate average
total_area = 0
num_entries = len(props)

for values in new_props.values():
    area = values['area']
    total_area += area

# Calculate the average height
average_area = total_area / num_entries
print(average_area)

7327.205398579321


In [14]:
import statistics

# Extract the areas from the bounding boxes
areas = [area for values in new_props.values() for area in [values['area']]]


# Calculate the lower and upper quartiles
q1 = statistics.quantiles(areas, n=4)[0]
q3 = statistics.quantiles(areas, n=4)[-1]

# Calculate the interquartile range (IQR)
iqr = q3 - q1

# Calculate the lower and upper bounds for outliers
lower_bound = q1 - (0.8 * iqr)
upper_bound = q3 + (0.8 * iqr)

# Remove outliers from the areas list
filtered_areas = [area for area in areas if lower_bound <= area <= upper_bound]
print(sorted(filtered_areas))
# Find the minimum and maximum areas from the filtered areas list
minimum_area = min(filtered_areas)
maximum_area = max(filtered_areas)
interval = (maximum_area - minimum_area) / 5
steps = [minimum_area + interval * i for i in range(6)]
#steps = [500, 2984.4, 5968.8, 8953.2, 11937.6, 14922.0]
print(steps)
# Find the closest fitting bounding box for each unique AngPos

step_filenames = []

# Specify the parent directory to create the step folders
parent_directory = imageData['path']

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)

for i, step in enumerate(steps):
    ang_positions = set(entry['AngPos'] for entry in new_props.values())

    # Initialize a list to store filenames for the current step
    current_step_filenames = []

    # Create the step directory
    step_directory = os.path.join(parent_directory, f'Step{i+1}')
    os.makedirs(step_directory)

    for ang_position in sorted(ang_positions):
        # Select the entries with the current 'AngPos' value
        selected_entries = [entry for entry in new_props.values() if entry['AngPos'] == ang_position]

        # Find the closest fitting bounding box for the current step and 'AngPos'
        closest_fitting_area = None
        min_distance = float('inf')

        for entry in selected_entries:
            area = entry['area']  # Calculate the height
            distance = abs(area - step)

            if distance < min_distance:
                closest_fitting_area = entry
                min_distance = distance

        # Append the filename for the closest fitting bounding box to the current step filenames list
        if closest_fitting_area is not None:
            filename = closest_fitting_area['filename']
            current_step_filenames.append(filename)

            # Move the file to the step directory
            source_path = os.path.join(imageData['path'], filename)
            destination_path = os.path.join(step_directory, filename)
            shutil.copy(source_path, destination_path)

    # Append the current step filenames to the overall list
    step_filenames.append(current_step_filenames)

# Print the step filenames
for i, filenames in enumerate(step_filenames):
    print(f"Step {i+1} filenames: {filenames}")




[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 600.0, 601.0, 602.0, 602.0, 602.0, 604.0, 605.0, 608.0, 608.0, 608.0, 609.0, 610.0, 610.0, 610.0, 610.0, 611.0, 611.0, 612.0, 613.0, 614.0, 615.0, 615.0, 616.0, 616.0, 618.0, 619.0, 619.0, 619.0, 619.0, 620.0, 620.0, 620.0, 621.0, 621.0, 622.0, 623.0, 624.0, 624.0, 624.0, 624.0, 624.0, 625.0, 626.0, 628.0, 628.0, 628.0, 629.0, 629.0, 630.0, 630.0, 631.0, 631.0, 631.0, 632.0, 632.0, 632.0, 632.0, 632.0, 633.0, 634.0, 635.0, 635.0, 635.0, 635.0, 635.0, 635.0, 636.0, 636.0, 637.0, 637.0, 637.0, 638.0, 638.0, 638.0, 638.0, 639.0, 640.0, 641.0, 641.0, 641.0, 641.0, 642.0, 642.0, 642.0, 642.0, 643.0, 643.0, 643.0, 644.0, 646.0, 646.0, 646.0, 646.0, 646.0, 646.0, 647.0, 647.0, 647.0, 647.0, 647.0, 648.0, 649.0, 649.0, 649.0, 650.0, 650.0, 651.0, 651.0, 652.0, 652.0, 653.0, 653.0, 653.0, 654.0, 654.0, 655.0, 655.0, 655.0, 655.0, 655.0, 655.0, 656.0, 656.

In [16]:
#Caros Version

# Number of closest areas to save
num_closest = 2

# Extract the areas
areas = [values['area'] for values in new_props.values()]

# Calculate lower and upper quartiles
q1 = statistics.quantiles(areas, n=4)[0]
q3 = statistics.quantiles(areas, n=4)[-1]

# Calculate interquartile range (IQR)
iqr = q3 - q1

# Calculate lower and upper bounds for outliers
lower_bound = q1 - (0.8 * iqr)
upper_bound = q3 + (0.8 * iqr)

# Remove outliers from areas list
filtered_areas = [area for area in areas if lower_bound <= area <= upper_bound]

# Find the minimum and maximum areas from filtered list
minimum_area = min(filtered_areas)
maximum_area = max(filtered_areas)
interval = (maximum_area - minimum_area) / 5


#steps = [minimum_area + interval * i for i in range(6)]
steps = [500, 2984.4, 5968.8, 8953.2, 11937.6, 14922.0]
print(steps)


# Specify the parent directory
parent_directory = imageData['path']

# Create the parent directory if it doesn't exist
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)

# Process each step
step_filenames = []
for i, step in enumerate(steps):
    ang_positions = set(entry['AngPos'] for entry in new_props.values())

    # Initialize a list to store filenames
    current_step_filenames = []

    # Create the step directory
    step_directory = os.path.join(parent_directory, f'Step{i+1}')
    os.makedirs(step_directory, exist_ok=True)

    for ang_position in sorted(ang_positions):
        # Select entries with the current 'AngPos' value
        selected_entries = [entry for entry in new_props.values() if entry['AngPos'] == ang_position]

        # Find the closest fitting areas for the current step and 'AngPos'
        closest_fitting_areas = []

        for entry in selected_entries:
            area = entry['area']
            distance = abs(area - step)
            closest_fitting_areas.append((distance, entry))

        # Sort by distance and keep only the top N entries
        closest_fitting_areas.sort(key=lambda x: x[0])  # Sort based on distance
        closest_fitting_areas = closest_fitting_areas[:num_closest]

        # Append the filenames for the closest fitting areas to the current step filenames list
        for _, closest_fitting_area in closest_fitting_areas:
            filename = closest_fitting_area['filename']
            current_step_filenames.append(filename)

            # Move the file to the step directory
            source_path = os.path.join(imageData['path'], filename)
            destination_path = os.path.join(step_directory, filename)
            shutil.copy(source_path, destination_path)

    # Append the current step filenames to the overall list
    step_filenames.append(current_step_filenames)

# Print the step filenames
for i, filenames in enumerate(step_filenames):
    print(f"Step {i+1} filenames: {filenames}")


[500, 2984.4, 5968.8, 8953.2, 11937.6, 14922.0]
Step 1 filenames: ['Scan1_00000000_0016.tif', 'Scan1_00000000_0012.tif', 'Scan1_00000001_0040.tif', 'Scan1_00000001_0041.tif', 'Scan1_00000002_0061.tif', 'Scan1_00000002_0066.tif', 'Scan1_00000003_0016.tif', 'Scan1_00000003_0066.tif', 'Scan1_00000004_0046.tif', 'Scan1_00000004_0031.tif', 'Scan1_00000005_0000.tif', 'Scan1_00000005_0001.tif', 'Scan1_00000006_0008.tif', 'Scan1_00000006_0007.tif', 'Scan1_00000007_0020.tif', 'Scan1_00000007_0021.tif', 'Scan1_00000008_0029.tif', 'Scan1_00000008_0062.tif', 'Scan1_00000009_0028.tif', 'Scan1_00000009_0060.tif', 'Scan1_00000010_0029.tif', 'Scan1_00000010_0028.tif', 'Scan1_00000011_0024.tif', 'Scan1_00000011_0054.tif', 'Scan1_00000012_0051.tif', 'Scan1_00000012_0019.tif', 'Scan1_00000013_0052.tif', 'Scan1_00000013_0021.tif', 'Scan1_00000014_0027.tif', 'Scan1_00000014_0028.tif', 'Scan1_00000015_0003.tif', 'Scan1_00000015_0037.tif', 'Scan1_00000016_0049.tif', 'Scan1_00000016_0050.tif', 'Scan1_00000017